# Réseau pré-entrainé : VGG16

In [2]:
import keras.applications as ka

In [3]:
model_VGG16 = ka.VGG16(include_top=False, weights='imagenet')
model_VGG16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [63]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
import keras.optimizers as ko

In [64]:
img_width = 150
img_height = 150

In [65]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), data_format="channels_last"))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64,name='first', input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1,name='second'))
model.add(Activation('sigmoid'))

In [66]:
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [67]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [68]:
import numpy as np
batch_size = 100
dir = 'data/features/'
generator1 = datagen.flow_from_directory(
        'data/train',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,  # this means our generator will only yield batches of data, no labels
        shuffle=False)  # our data will be in order, so all first 1000 images will be cats, then 1000 dogs

Found 2000 images belonging to 2 classes.


In [33]:
features_train = model.predict_generator(generator1, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)
features_train[:5]

array([[0.561924  ],
       [0.5422448 ],
       [0.5279867 ],
       [0.53950953],
       [0.5504598 ]], dtype=float32)

In [25]:
np.save(open(dir+'features_train.npy', 'wb'), features_train)

In [69]:
generator2 = datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

Found 800 images belonging to 2 classes.


In [32]:
features_validation = model.predict_generator(generator2, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)
features_validation[:5]

array([[0.5417565 ],
       [0.5270453 ],
       [0.5087023 ],
       [0.52419484],
       [0.5364618 ]], dtype=float32)

In [28]:
np.save(open(dir+'features_validation.npy', 'wb'), features_validation)

In [70]:
train_labels = np.array([0] * 1000 + [1] * 1000)
train_labels[995:1005]

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

In [71]:
train_data = np.load(open('data/features/features_train.npy' , 'rb'))
# the features were saved in order, so recreating the labels is easy
train_labels = np.array([0] * 1000 + [1] * 1000)

validation_data = np.load(open(dir+'features_validation.npy', 'rb'))
validation_labels = np.array([0] * 400 + [1] * 400)

train_data.shape

(2000, 1)

In [78]:
#model_VGG
model.add(Flatten(input_shape=(train_data.shape[1:])))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

ValueError: Input 0 is incompatible with layer flatten_14: expected min_ndim=3, found ndim=2